In [1]:
"""
Imports
The core
"""
import MAG_Util as MAG_Util
import MAG_Manager_Dataframe as MAG_Manager_Dataframe
import MAG_Graph_Layered as MAG_Graph_Layered


Generate the sample graph for power grid

In [2]:
"""
Generate the random graph for a power grid system
If power is sent from pointA to pointB then
- u = pointA
- v = pointB
- w = cost (such as power loss)
"""

# generate the graph as a random one
import Generator_Graph as Generator_Graph
current_edges_citations = Generator_Graph.generated_graph_simple(
    argv_count_vertices=100,
    argv_count_edges=300,
    argv_flag_directed=True,
    argv_weight_type="random",
    argv_weight_range=[1,100],
    argv_random_seed=888
    )

# have it output into a file
filename_graph = "Sample_PowerGrid.csv"
file_graph = open(filename_graph, "w+")
# header
file_graph.write("source,destination,cost")
file_graph.write("\n")
# add in the rows 1 by 1
for current_edge in current_edges_citations:
    file_graph.write(current_edge + "\n")
file_graph.close()

In [3]:
# the dataset filename
dict_filenames_dataset = {}
dict_filenames_dataset["PowerGrid"] = "Sample_PowerGrid.csv"

# manager_dataframe to load in the data
manager_dataframe = MAG_Manager_Dataframe.Manager_Dataframe(argv_filename=dict_filenames_dataset)
manager_dataframe.load_dataframes()

In [4]:
# get the dataframe
df_power = manager_dataframe.get_dataframe(argv_id="PowerGrid")

In [5]:
import cudf

In [6]:
# the generator IDs
current_generator_IDs = [10,20,30,40,50]

# create a new vertex as the source to these generators at a cost of 0
# current_source_apex = max(df_power["source"].max(),df_power["destination"].max()) + 1
current_source_apex = -1

# create a quick dataframe for the additional edges
temp_dict = {}
temp_dict["source"] = []
temp_dict["destination"] = []
temp_dict["cost"] = []
for current_generator_ID in current_generator_IDs:
    temp_dict["source"].append(current_source_apex)
    temp_dict["destination"].append(current_generator_ID)
    temp_dict["cost"].append(0)
temp_frame = cudf.DataFrame(temp_dict)
# then concat it
df_power = cudf.concat([df_power, temp_frame], ignore_index = True)
    
# get the head to check
print(df_power.tail(10))

     source  destination  cost
295      51           65    79
296      46           18    64
297      53           12    19
298      79           64    30
299      40           42    45
300      -1           10     0
301      -1           20     0
302      -1           30     0
303      -1           40     0
304      -1           50     0


With this, we now want to find the minimal cost from each generator to every possible point

In [7]:
# the variables
colname_source = "source"
colname_destination = "destination"
colname_weight = "cost"

# make the graph object
current_graph = MAG_Graph_Layered.Graph_Layered(
    argv_dataframe=df_power,
    argv_source=colname_source, argv_destination=colname_destination, argv_weight=[colname_weight]
    )
# generate the graph
current_graph.generate_graph()
print(current_graph.get_stat_weights())

# let say we want the relationship between these papers
starting_point = current_source_apex

# run dijkstra
# without the end (single source only)
table_pred_distance, timer_dijkstra = current_graph.run_dijsktra(
    argv_start=str(starting_point), argv_path=False, argv_layer=False
    )

# print out the information
print(table_pred_distance)

{'min': 0, 'max': 100, 'mean': 51.501639344262294, 'median': 54, 'zero': 0}
     distance vertex predecessor
0       101.0     36           3
1        92.0     37          13
2        83.0     41          72
3       100.0     43          24
4       105.0     48          80
..        ...    ...         ...
96      103.0     39          87
97      130.0     44          51
98       98.0     47          81
99      202.0     63          56
100       0.0     -1          -1

[101 rows x 3 columns]
